# Few-shot Learning on DIRG

We demonstrate the FS learning framework on the dataset `DIRG`, which has fewer classes, a higher sampling rate and more channels than `CWRU`. It turns out `DIRG` is significantly harder. Here are some points worth further study to improve the performance:

- characteristics of the feature: STFS window length, spectral patch shape
- choose of channels: 'A1' and/or 'A2'. A pre-input layer is necessary if the number of channels isn't 3

The best performance obtained using 20% of data for training is ~ 70% after the fine tuning.

In [1]:
%xmode minimal

import os
import json

# Turn off logging for TF
import logging
# tf.get_logger().setLevel(logging.ERROR)
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"  # disable GPU devices
os.environ["TFDS_DATA_DIR"] = os.path.expanduser("~/tensorflow_datasets")  # default location of tfds database

import os
# os.environ["KERAS_BACKEND"] = "tensorflow"
os.environ["KERAS_BACKEND"] = "jax"
# os.environ["KERAS_BACKEND"] = "torch"

import keras
from keras import layers, models, ops

import tensorflow as tf
import tensorflow_datasets as tfds

import librosa
import librosa.display

import numpy as np
from matplotlib import pyplot as plt

from pathlib import Path

# from IPython.display import Audio

# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

Exception reporting mode: Minimal


In [2]:
import dpmhm
# dpmhm.datasets.get_dataset_list()

from dpmhm.datasets import preprocessing, feature, utils, transformer, query_parameters

In [3]:
ds_name = 'DIRG'

outdir = Path(f'/home/han/tmp/dpmhm/few-shot/{ds_name}')
os.makedirs(outdir, exist_ok=True)

In [4]:
query_parameters(ds_name)

{'signal': {'A1': 3, 'A2': 3},
 'sampling_rate': [51200, 102400],
 'keys': {'FaultComponent': {'InnerRing', 'Roller'},
  'FaultSize': {0, 150, 250, 450}},
 'filters': {'RotatingSpeed': {100, 200, 300, 400, 500},
  'NominalLoadForce': {0, 1000, 1400, 1800}},
 'type': 'initiated+failure',
 'split': ['vibration', 'endurance']}

## Load a dataset

In [5]:
_func = lambda x, sr: feature.spectral_features(
    x, sr, 'spectrogram',
    # n_mfcc=256,
    time_window=0.025, hop_step=0.0125,
    # n_fft=512,
    normalize=False, to_db=True)[0]

compactor_kwargs = dict(
    # # CWRU
    # channels=[],
    # keys=['FaultLocation', 'FaultComponent', 'FaultSize'],
    # DIRG
    # channels=['A1', 'A2'],
    channels=['A1'],
    keys=['FaultComponent', 'FaultSize'],
)

window_kwargs = dict(
    # window_size=(128,64), 
    # hop_size=(128,64),
    window_size=(64,64), 
    hop_size=(64,64),
    # hop_size=(32,32)    
)

ds0, full_label_dict = dpmhm.datasets.spectral_window_pipeline(
    ds_name, 
    split='variation',
    spectral_feature=_func,
    compactor_kwargs=compactor_kwargs,
    window_kwargs=window_kwargs,
)

print('Full dictionary of labels:', full_label_dict)

label_index_dict = preprocessing.get_label_mapping(list(full_label_dict.keys()))
print('Integer index of labels:', label_index_dict)

Full dictionary of labels: {'6c2972e6362a149d': ['InnerRing', '450'], 'f1ec5257d3156fa9': ['Roller', '150'], '536057fae7e77d53': ['None', '0'], 'e67b723eb0cefad6': ['Roller', '250'], '3f589794bedff28d': ['InnerRing', '250'], '0e585b7656ed7f4c': ['InnerRing', '150'], '8f6697984c79c128': ['Roller', '450']}
Integer index of labels: {'6c2972e6362a149d': 1, 'f1ec5257d3156fa9': 2, '536057fae7e77d53': 3, 'e67b723eb0cefad6': 4, '3f589794bedff28d': 5, '0e585b7656ed7f4c': 6, '8f6697984c79c128': 7}


### Split for few-shot learning with OOD

Jump directly to the next section "Export and reload..." if a dataset has already be produced.

In [6]:
preproc = preprocessing.get_mapping_supervised(list(full_label_dict.keys()))

ds1 = utils.restore_shape(
    ds0.map(preproc, num_parallel_calls=tf.data.AUTOTUNE),
    key=0
)

In [7]:
# OOD labels
labels_ood = [1]
labels = [l for l in label_index_dict.values() if l not in labels_ood]

# Extract OOD samples and concatenate into a single OOD category
# use key=1 here because `ds1` is a tuple dataset and 1 is the index of the label field
foo = utils.extract_by_category(ds1, labels_ood, key=1)  
ds_ood = None
for k, dv in foo.items():
    try:
        ds_ood = ds_ood.concatenate(dv)
    except:
        ds_ood = dv

# Few-shot split
splits = {'train':0.2, 'val':0.7, 'test':0.1}
ds_split = utils.split_dataset(ds1, splits=splits, labels=labels, key=1)

#### Export and reload the preprocessed dataset 

For a better performance

In [6]:
dbdir = outdir / utils.md5_encoder([compactor_kwargs, window_kwargs])
# Bug in Tensorflow: folder name containing '[ ]'
# dbdir = outdir/f"channels[{compactor_kwargs['channels']}]_windowsize[{window_kwargs['window_size']}]"

os.makedirs(dbdir, exist_ok=True)

In [9]:
for k, dv in ds_split.items():
    dv.save(str(dbdir/k))

ds_ood.save(str(dbdir/'ood'))

In [7]:
ds_split = {}

for k in ['train', 'val', 'test']:
    ds_split[k] = tf.data.Dataset.load(str(dbdir/k))

ds_ood = tf.data.Dataset.load(str(dbdir/'ood'))

print(ds_ood.element_spec, ds_ood.cardinality())

# ds_split['train'].cardinality()

(TensorSpec(shape=(64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None)) tf.Tensor(3264, shape=(), dtype=int64)


In [8]:
list(ds_ood.take(1))

[(<tf.Tensor: shape=(64, 64, 3), dtype=float32, numpy=
  array([[[46.202293, 62.192078, 58.752598],
          [37.077477, 68.385185, 64.99098 ],
          [37.469414, 68.46131 , 65.06549 ],
          ...,
          [38.334637, 68.47639 , 64.78662 ],
          [37.787514, 68.39858 , 64.78197 ],
          [37.9596  , 68.31246 , 64.82229 ]],
  
         [[45.900124, 61.477142, 58.044533],
          [34.768158, 66.16981 , 62.738518],
          [35.287384, 66.20686 , 62.83062 ],
          ...,
          [36.609238, 66.22987 , 62.547684],
          [35.634476, 66.151245, 62.536148],
          [35.66985 , 66.06192 , 62.581184]],
  
         [[45.232895, 59.29322 , 55.8773  ],
          [26.86792 , 58.688934, 55.0765  ],
          [27.811806, 58.5095  , 55.254154],
          ...,
          [31.445742, 58.571144, 54.966766],
          [28.559397, 58.508118, 54.881035],
          [27.7387  , 58.38472 , 54.955673]],
  
         ...,
  
         [[40.85623 , 34.909885, 47.234863],
          [38.98

## Train a VGGish network

In [9]:
batch_size = 64
shuffle_size = max(1000, ds_split['train'].cardinality())

ds_train = ds_split['train']\
    .shuffle(shuffle_size, reshuffle_each_iteration=True)\
    .batch(batch_size, drop_remainder=True)\
    .prefetch(tf.data.AUTOTUNE)
ds_val = ds_split['val'].batch(batch_size, drop_remainder=True)
ds_test = ds_split['test'].batch(batch_size, drop_remainder=True)

ds_ood_test = ds_ood.batch(batch_size, drop_remainder=True)

n_classes = len(full_label_dict) + 1

input_shape = ds_split['train'].element_spec[0].shape

if input_shape[-1] != 3:
    # If the number of channels in the orignal data isn't 3, use a first layer to adapt to the base model
    input_model = models.Sequential([
        layers.Input(shape=input_shape, name='input'),
        layers.Conv2D(3, kernel_size=(1,1), activation=None, padding='same')
    ])
    # input_shape = input_model(layers.Input(input_shape)).shape
    input_shape1 = (*input_shape[:-1], 3)
else:
    input_model = models.Sequential([
        layers.Input(shape=input_shape, name='input'),
    ])
    input_shape1 = input_shape

In [10]:
from keras.applications import VGG16, resnet

base_model = VGG16(include_top=False, weights='imagenet', input_shape=input_shape1, pooling='max')
# base_model = resnet.ResNet50(include_top=False, weights='imagenet', input_shape=input_shape1, pooling='max')

base_model.trainable = False

In [12]:
x = layers.Input(input_shape)

adapt_model = models.Sequential([
    layers.Flatten(name="flatten"),
    # layers.Dense(256, activation="relu", name="fc1"),
    layers.Dense(4096, activation="relu", name="fc1"),
    layers.BatchNormalization(),
    # layers.Dense(128, activation="relu", name="fc2"),
    layers.Dense(4096, activation="relu", name="fc2"),
    layers.BatchNormalization(),
    layers.Dense(n_classes, activation=None, name="predictions")
])

y = adapt_model(base_model(input_model(x)))

In [13]:
model = models.Model(x, y)

from_logits = 'softmax' not in str(model.layers[-1].get_layer('predictions').activation)

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=from_logits),
    metrics=['accuracy'],
)

In [14]:
history = model.fit(
    ds_train,
    validation_data=ds_val.take(10),
    epochs=20,
    callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=3),
)

Epoch 1/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 40s 635ms/step - accuracy: 0.2962 - loss: 5.0769 - val_accuracy: 0.5406 - val_loss: 2.5787
Epoch 2/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 39s 624ms/step - accuracy: 0.5170 - loss: 1.3911 - val_accuracy: 0.5109 - val_loss: 2.2224
Epoch 3/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 38s 619ms/step - accuracy: 0.5728 - loss: 1.2863 - val_accuracy: 0.5047 - val_loss: 1.9262
Epoch 4/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 38s 613ms/step - accuracy: 0.5929 - loss: 1.1848 - val_accuracy: 0.4547 - val_loss: 2.0364
Epoch 5/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 37s 612ms/step - accuracy: 0.6211 - loss: 1.1043 - val_accuracy: 0.6641 - val_loss: 1.0572
Epoch 6/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 38s 617ms/step - accuracy: 0.6842 - loss: 0.9084 - val_accuracy: 0.6219 - val_loss: 1.4327
Epoch 7/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 38s 619ms/step - accuracy: 0.7033 - loss: 0.8741 - val_accuracy: 0.6141 - val_loss: 1.4032
Epoch 8/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 38s 615ms/step - accuracy: 0.7057 - loss: 0.8914 - val_accu

In [18]:
history = model.fit(
    ds_train,
    validation_data=ds_val.take(10),
    epochs=20,
    callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=5),
)

Epoch 1/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 38s 615ms/step - accuracy: 0.8655 - loss: 0.4330 - val_accuracy: 0.6109 - val_loss: 1.4836
Epoch 2/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 38s 619ms/step - accuracy: 0.8683 - loss: 0.3869 - val_accuracy: 0.8328 - val_loss: 0.5917
Epoch 3/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 38s 615ms/step - accuracy: 0.8867 - loss: 0.3569 - val_accuracy: 0.6672 - val_loss: 1.4470
Epoch 4/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 38s 619ms/step - accuracy: 0.8894 - loss: 0.3431 - val_accuracy: 0.6328 - val_loss: 1.5205
Epoch 5/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 38s 617ms/step - accuracy: 0.8784 - loss: 0.4009 - val_accuracy: 0.8188 - val_loss: 0.8124
Epoch 6/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 38s 626ms/step - accuracy: 0.9045 - loss: 0.3068 - val_accuracy: 0.7750 - val_loss: 0.8625
Epoch 7/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 38s 615ms/step - accuracy: 0.9189 - loss: 0.2760 - val_accuracy: 0.6656 - val_loss: 1.6977
Epoch 7: early stopping


Performance of the trained model on test data:

In [19]:
model.evaluate(ds_test)

30/30 ━━━━━━━━━━━━━━━━━━━━ 14s 477ms/step - accuracy: 0.5940 - loss: 1.9041


[2.0414507389068604, 0.546875]

On the contrary, on OOD data completely failed. 

In [20]:
model.evaluate(ds_ood_test)

51/51 ━━━━━━━━━━━━━━━━━━━━ 24s 479ms/step - accuracy: 0.0000e+00 - loss: 11.8054


[11.648832321166992, 0.0]

### Fine tuning

Fine tuning in the few-shot learning scenario may still improve the model's performance, despite the insufficient training data.

In [21]:
base_model.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=from_logits),
    metrics=['accuracy'],
)

In [22]:
history = model.fit(
    ds_train,
    validation_data=ds_val.take(10),
    epochs=2,
    callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=3),
)

Epoch 1/2
61/61 ━━━━━━━━━━━━━━━━━━━━ 138s 2s/step - accuracy: 0.9179 - loss: 0.2620 - val_accuracy: 0.7828 - val_loss: 0.9710
Epoch 2/2
61/61 ━━━━━━━━━━━━━━━━━━━━ 136s 2s/step - accuracy: 0.9498 - loss: 0.1710 - val_accuracy: 0.8234 - val_loss: 0.7939


In [23]:
model.evaluate(ds_test)

30/30 ━━━━━━━━━━━━━━━━━━━━ 14s 477ms/step - accuracy: 0.7230 - loss: 1.2123


[1.307074785232544, 0.6854166388511658]

In [24]:
model.evaluate(ds_ood_test)

51/51 ━━━━━━━━━━━━━━━━━━━━ 24s 472ms/step - accuracy: 0.0000e+00 - loss: 11.1884


[11.05380916595459, 0.0]

### Adaptation on OOD data

We may adapt the model on OOD data, however this may incur the catastrophic forget.

In [36]:
# Split the OOD data
ds_ood_split = utils.split_dataset(ds_ood, {'train':0.2, 'val':0.7, 'test':0.1}, key=1)

batch_size = 16

ds_ood_train = ds_ood_split['train']\
    .shuffle(1000, reshuffle_each_iteration=True)\
    .batch(batch_size, drop_remainder=True)\
    .prefetch(tf.data.AUTOTUNE)
ds_ood_val = ds_ood_split['val'].batch(batch_size, drop_remainder=True)
ds_ood_test = ds_ood_split['test'].batch(batch_size, drop_remainder=True)

In [44]:
base_model.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=from_logits),
    metrics=['accuracy'],
)

In [45]:
history = model.fit(
    ds_ood_train,
    validation_data=ds_ood_val,
    epochs=10,
    callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=3),
)

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.0000e+00 - loss: 27.8739 - val_accuracy: 0.0000e+00 - val_loss: 3.5991
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.2042 - loss: 2.8744 - val_accuracy: 0.4805 - val_loss: 2.8254
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.4021 - loss: 2.1012 - val_accuracy: 0.5234 - val_loss: 0.7230
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.5021 - loss: 0.8861 - val_accuracy: 0.4961 - val_loss: 0.7041
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.3688 - loss: 0.7533 - val_accuracy: 0.5000 - val_loss: 0.6899
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.5500 - loss: 0.7189 - val_accuracy: 0.4922 - val_loss: 0.6937
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.6542 - loss: 0.6928 - val_accuracy: 0.5000 - val_loss: 0.7093
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.5354 - loss: 0.7058 - val_accuracy: 0.5195 - val_loss: 0.7011
Epoch 8

In [47]:
model.evaluate(ds_ood_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.5625 - loss: 0.6792


[0.6783491373062134, 0.5625]

In [46]:
model.evaluate(ds_test)

7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 346ms/step - accuracy: 0.0000e+00 - loss: 11.6353


[11.451746940612793, 0.0]

## Alternative of the splitting scheme

Here we applied the split step first. It's also possible (and equivalent) to apply the preprocessing step which converts string labels to integer indexes after the split, as shown below.

In [ ]:
# Split for few-shot learning with OOD

# OOD labels
# labels_ood = ['0881fa248109963a']  # CWRU: the 3 channels data contain no normal samples
labels_ood = ['55503c950ed81973']  # ['FanEnd', 'Ball', '0.014']
# Other labels
labels = [l for l in full_label_dict if l not in labels_ood]
# labels = list(full_label_dict.keys())

# Extract OOD samples and concatenate into a single OOD category
foo = utils.extract_by_category(ds0, labels_ood)
for k, dv in foo.items():
    try:
        ds_ood = ds_ood.concatenate(dv)
    except:
        ds_ood = dv

# Few-shot split
splits = {'train':0.2, 'val':0.7, 'test':0.1}
ds_split = utils.split_dataset(ds0, splits=splits, labels=labels)

In [ ]:
preproc = preprocessing.get_mapping_supervised(list(full_label_dict.keys()))

batch_size = 64

ds_train = ds_split['train']\
    .map(preproc, num_parallel_calls=tf.data.AUTOTUNE)\
    .shuffle(1000, reshuffle_each_iteration=True)\
    .batch(batch_size, drop_remainder=True)\
    .prefetch(tf.data.AUTOTUNE)
ds_val = ds_split['val']\
    .map(preproc, num_parallel_calls=tf.data.AUTOTUNE)\
    .batch(batch_size, drop_remainder=True)
ds_test = ds_split['test']\
    .map(preproc, num_parallel_calls=tf.data.AUTOTUNE)\
    .batch(batch_size, drop_remainder=True)

In [ ]:
with open(outdir/'full_labels.json', 'w') as fp:
    json.dump(full_label_dict,fp)

with open(outdir/'label_mapping.json', 'w') as fp:
    json.dump(label_index_dict,fp)

# dp_split = {}
# for k, dv in ds_split.items():
#     dp_split[k] = utils.restore_shape(
#         ds_split[k].map(preproc, num_parallel_calls=tf.data.AUTOTUNE)
#     )
# )
# # ds_size = utils.get_dataset_size(ds_window)

# EOF